In [1]:
import os
import sys
import re
import pandas as pd
import commands
import ast
from StringIO import StringIO
import itertools
import pyspark.sql.functions
from pyspark.sql.functions import col
from datetime import datetime
from collections import Counter
import re
import numpy as np
#from pyspark.sql import SparkSession
spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
if not os.path.isdir(spark_home):
    raise ValueError('SPARK_HOME environment variable is not a directory')
if not os.path.isdir(os.path.join(spark_home, 'python')):
    raise ValueError('SPARK_HOME directory does not contain python')
sys.path.insert(0, os.path.join(spark_home, 'python'))
pylib_list = (item for item in os.listdir(os.path.join(spark_home, 'python/lib/'))
              if re.match(r'py4j-\d+(\.\d+)+-src\.zip\Z', item)
              )
try:
    py4j_file = max(pylib_list)
    py4j = os.path.join(spark_home, os.path.join('python/lib', py4j_file))
except ValueError:
    raise ValueError(
        'Could not find py4j'
    )
sys.path.insert(0, py4j)

spark_release_file = spark_home + "/RELEASE"
if os.path.exists(spark_release_file) and "Spark" in  open(spark_release_file).read():
    pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", " --master yarn-client \
                                         --executor-memory 4g --executor-cores 5 --driver-memory 16g"
                                        )
    if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
    os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

with open(os.path.join(spark_home, 'python/pyspark/shell.py')) as f:
    code = compile(f.read(), os.path.join(spark_home, 'python/pyspark/shell.py'), 'exec')
    exec(code)


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.2.6.5.0-292
      /_/

Using Python version 2.7.15 (default, May  1 2018 23:32:55)
SparkSession available as 'spark'.


# Convert DDLs to csv using DESDDL2CSV.sh and use them as inputs

In [71]:
df_today=pd.read_csv("/s/latluri1/DES/DES_20190430_processed.csv",delimiter=",").apply(lambda x: x.astype(str).str.upper())
df_yesterday=pd.read_csv("/s/latluri1/DES/DESscript_20190104_processed.csv",delimiter=",").apply(lambda x: x.astype(str).str.upper())
df_yesterday2=df_yesterday.apply(lambda x: x.astype(str).str.lower())
df_today2=df_today.apply(lambda x: x.astype(str).str.lower())


In [72]:
tab_list=['account','activityparty','annotation','appointment','businessunit','calendar','calendarrule','contact','cxlvhlp_chatactivity','cxlvhlp_chatqueuestatistic','cxlvhlp_surveyitem','email','fax','gcct_accountresponsibleagent','gcct_additionalsymptomcodes','gcct_addportalmessage','gcct_arbitrationclaimprocessing','gcct_buybackevaluationmilestones','gcct_caseassignment','gcct_caseclassification','gcct_casedispositiontype','gcct_coachback','gcct_country','gcct_customersatisfactiontools','gcct_delegationofauthority','gcct_demandltrtpsmclaimsprocessing','gcct_doaprogramcode','gcct_documentcustomerrecontact','gcct_engine','gcct_executive','gcct_executiveliaison','gcct_fieldinvolvementassistancerequest','gcct_fmcccontract','gcct_fulfillmentvendor','gcct_genericinformation','gcct_genericinformationtopic','gcct_generictopic','gcct_goodfaithreview','gcct_knownpartsdelay','gcct_lemonlawcriteria','gcct_loanercost','gcct_loyaltyassistance','gcct_loyaltycostdetails','gcct_loyaltyprogramcode','gcct_material','gcct_materialrequest','gcct_materialrequestdetail','gcct_offer','gcct_offerdetail','gcct_onlinegaragevehicles','gcct_partsorderstatus','gcct_partsordersystems','gcct_partssmeassistancerequest','gcct_pcarequest','gcct_priordealerdecision','gcct_qaincident','gcct_qamonitoring','gcct_queuepriority','gcct_ravprocessingmilestones','gcct_reactdata','gcct_reacttransmission','gcct_reasoncode','gcct_recall','gcct_region','gcct_rentaldetail','gcct_slaconfiguration','gcct_sms','gcct_socialmediaconversation','gcct_socialmediamessage','gcct_socialmediaprofile','gcct_specialloanercode','gcct_state','gcct_surveyconfiguration','gcct_symptomclassification','gcct_tasktype','gcct_technicalassistancerequest','gcct_testdriverequest','gcct_timezone','gcct_userdelegationofauthority','gcct_vehicle','gcct_vehiclebrand','gcct_vehicleclass','gcct_vehicleloyaltyallowance','gcct_vehicleoffroad','gcct_vehicleowner','gcct_warrantyhistory','gcct_warrantyloanerpartsdelay','gcct_warrantyloanerrequest','gcct_warrantyloanerrequesthistory','globaloptionsetmetadata','incident','incidentresolution','letter','msdyn_answer','msdyn_question','msdyn_questionresponse','msdyn_survey','msdyn_surveyinvite','msdyn_surveyresponse','optionsetmetadata','phonecall','queue','queueitem','sla','slaitem','slakpiinstance','socialactivity','socialprofile','statemetadata','statusmetadata','systemuser','task','team','teammembership','territory']
def priority(r):
    if(r in tab_list):
        return("True")
    else:
        return("False")
    
    
def get_decimal(a):
    if(a['DATA_TYPE']=='decimal'):
        if np.logical_not(pd.isnull(a['NUMERIC_PRECISION'])):
            if np.logical_not(pd.isnull(a['NUMERIC_SCALE'])):
                return(str(a['DATA_TYPE'])+"("+str(int(float(a['NUMERIC_PRECISION'])))+","+str(int(float(a['NUMERIC_SCALE'])))+")")
            else:
                return(str(a['DATA_TYPE'])+"("+str(int(float(a['NUMERIC_PRECISION'])))+")")
        else:
            return(a['DATA_TYPE'])
    else:
        return(a['DATA_TYPE'])


tab_removed_1=set(list(df_yesterday2.TABLE_NAME)).difference(set(list(df_today2.TABLE_NAME)))
tab_added_1=set(list(df_today2.TABLE_NAME)).difference(set(list(df_yesterday2.TABLE_NAME)))


df_today2.loc[:,'DATA_TYPE']=df_today2[['DATA_TYPE','NUMERIC_PRECISION','NUMERIC_SCALE']].apply(lambda r: get_decimal(r), axis=1)
df_yesterday2.loc[:,'DATA_TYPE']=df_yesterday2[['DATA_TYPE','NUMERIC_PRECISION','NUMERIC_SCALE']].apply(lambda r: get_decimal(r), axis=1)


In [88]:
final=df_today2.merge(df_yesterday2,indicator = True, how='outer',on=['TABLE_NAME','COLUMN_NAME'],suffixes=('_new','_old'))
final.rename(columns={'_merge':'Change'}, inplace=True)

In [89]:
final["Change"]=final.Change.replace("left_only","Column added").replace("right_only","Column removed").replace("both","No change")
final.loc[(final['DATA_TYPE_old'] != final['DATA_TYPE_new']) & (final.Change =="No change"),"Change"]="Dtype_change"
#final.loc[(final['Numeric_precision_scale_old'] == final['Numeric_precision_scale_new']) & (final.Change =="no change"),"Change"]="dtype_change"

In [90]:
final.loc[final.TABLE_NAME.isin(tab_removed_1),"Change"]="Table removed"
final.loc[final.TABLE_NAME.isin(tab_added_1),"Change"]="Table Added"

In [98]:
result=final.loc[final.Change !="No change",["TABLE_NAME","COLUMN_NAME","DATA_TYPE_new","DATA_TYPE_old","Change"]]
result["priority"]=result["TABLE_NAME"].apply(lambda r: priority(r))


In [94]:

if len(result) >0:
    result.to_excel("Comp_2_ddls.xlsx",index=False)
    commands.getstatusoutput("echo -e \"Compared two ddls and observed the output attached\"| mailx -s 'Two ddls comparison' -a \"Comp_2_ddls.xlsx\" latluri1@ford.com  ")	
